لتريع الحسابات ويعمل فى كولاب

https://huggingface.co/docs/diffusers/optimization/xformers













https://github.com/facebookresearch/xformers#installing-xformers









https://huggingface.co/docs/diffusers/optimization/memory#memory-efficient-attention













https://huggingface.co/docs/diffusers/optimization/fp16

In [ ]:
!pip install xformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import xformers

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# تمكين xFormers
model = xformers.optimize(model)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# تشغيل الاستدلال
with torch.no_grad():
    outputs = model(**inputs)

# استخراج النص الناتج
output_text = tokenizer.decode(outputs.logits.argmax(dim=-1)[0], skip_special_tokens=True)
print(output_text)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import xformers
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model(**inputs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
model_with_xformers = xformers.optimize(model)

# تشغيل الاستدلال مع xFormers
start_time = time.time()
with torch.no_grad():
    outputs_with_xformers = model_with_xformers(**inputs)
end_time = time.time()
time_with_xformers = end_time - start_time

# استخراج النص الناتج
output_text_no_xformers = tokenizer.decode(outputs_no_xformers.logits.argmax(dim=-1)[0], skip_special_tokens=True)
output_text_with_xformers = tokenizer.decode(outputs_with_xformers.logits.argmax(dim=-1)[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

print(f"Output with xFormers: {output_text_with_xformers}")
print(f"Time with xFormers: {time_with_xformers:.6f} seconds")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import xformers
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model(**inputs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
model_with_xformers = model
if hasattr(model_with_xformers, 'half'):
    model_with_xformers = model_with_xformers.half()  # استخدام الدقة النصفية لتحسين الأداء

# تشغيل الاستدلال مع xFormers
start_time = time.time()
with torch.no_grad():
    outputs_with_xformers = model_with_xformers(**inputs)
end_time = time.time()
time_with_xformers = end_time - start_time

# استخراج النص الناتج
output_text_no_xformers = tokenizer.decode(outputs_no_xformers.logits.argmax(dim=-1)[0], skip_special_tokens=True)
output_text_with_xformers = tokenizer.decode(outputs_with_xformers.logits.argmax(dim_-1)[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

print(f"Output with xFormers: {output_text_with_xformers}")
print(f"Time with xFormers: {time_with_xformers:.6f} seconds")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model(**inputs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء إذا كانت مدعومة
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers(**inputs.to('cuda'))
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers.logits.argmax(dim=-1)[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers.logits.argmax(dim=-1)[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time
import xformers.ops

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model(**inputs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

# تشغيل الاستدلال مع xFormers
start_time = time.time()
with torch.no_grad():
    outputs_with_xformers = model_with_xformers(**inputs.to('cuda'))
end_time = time.time()
time_with_xformers = end_time - start_time

# استخراج النص الناتج
output_text_no_xformers = tokenizer.decode(outputs_no_xformers.logits.argmax(dim=-1)[0], skip_special_tokens=True)
output_text_with_xformers = tokenizer.decode(outputs_with_xformers.logits.argmax(dim=-1)[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

print(f"Output with xFormers: {output_text_with_xformers}")
print(f"Time with xFormers: {time_with_xformers:.6f} seconds")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time
import xformers.ops

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], max_length=50)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

# تشغيل الاستدلال مع xFormers
start_time = time.time()
with torch.no_grad():
    outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), max_length=50)
end_time = time.time()
time_with_xformers = end_time - start_time

# استخراج النص الناتج
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)
output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

print(f"Output with xFormers: {output_text_with_xformers}")
print(f"Time with xFormers: {time_with_xformers:.6f} seconds")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.eos_token_id

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=50, pad_token_id=pad_token_id)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), max_length=50, pad_token_id=pad_token_id)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# أو يمكنك إضافة رمز padding جديد إلى المحول
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# model.resize_token_embeddings(len(tokenizer))

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=50, pad_token_id=pad_token_id)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), max_length=50, pad_token_id=pad_token_id)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 5,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 0.9,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 5,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 0.9,
    "do_sample": True,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 5,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 0.9,
    "do_sample": True,
    "repetition_penalty": 1.2,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 5,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 0.9,
    "do_sample": True,
    "repetition_penalty": 1.2,
    "no_repeat_ngram_size": 3,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

احسن كود ف الصفحة

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # أو أي نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 5,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 0.9,
    "do_sample": True,
    "repetition_penalty": 1.2,
    "no_repeat_ngram_size": 3,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

كود جيد

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # يمكن اختيار نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 5,
    "temperature": 0.7,
    "top_k": 50,
    "top_p": 0.9,
    "do_sample": True,
    "repetition_penalty": 1.2,
    "no_repeat_ngram_size": 3,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

كود رائع

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # يمكن اختيار نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 5,
    "temperature": 0.8,
    "top_k": 50,
    "top_p": 0.9,
    "do_sample": True,
    "repetition_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

شغال جيد

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # يمكن اختيار نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 5,
    "temperature": 0.8,
    "top_k": 50,
    "top_p": 0.9,
    "do_sample": True,
    "repetition_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model.half()
    # نقل النموذج إلى GPU إذا كانت متاحة
    model_with_xformers = model_with_xformers.to('cuda')

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # يمكن اختيار نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 5,
    "temperature": 1.0,  # زيادة تنوع النص
    "top_k": 50,
    "top_p": 0.95,  # تحسين جودة النص
    "do_sample": True,
    "repetition_penalty": 2.0,  # تقليل التكرار
    "no_repeat_ngram_size": 3,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # يمكن اختيار نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 7,
    "temperature": 1.2,  # زيادة تنوع النص
    "top_k": 50,
    "top_p": 0.95,  # تحسين جودة النص
    "do_sample": True,
    "repetition_penalty": 2.5,  # تقليل التكرار
    "no_repeat_ngram_size": 3,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # يمكن اختيار نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 50,
    "num_beams": 5,
    "temperature": 1.2,  # زيادة تنوع النص
    "top_k": 50,
    "top_p": 0.95,  # تحسين جودة النص
    "do_sample": True,
    "repetition_penalty": 2.0,  # تقليل التكرار
    "no_repeat_ngram_size": 3,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "gpt2"  # يمكن اختيار نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "8+10=؟"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 100,  # زيادة طول النص الناتج
    "num_beams": 5,
    "temperature": 1.2,  # زيادة تنوع النص
    "top_k": 50,
    "top_p": 0.95,  # تحسين جودة النص
    "do_sample": True,
    "repetition_penalty": 2.0,  # تقليل التكرار
    "no_repeat_ngram_size": 3,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
Qwen/Qwen2.5-Coder-0.5B-Instruct

شغال جيد جدا

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

# تحميل النموذج والمحول (tokenizer)
model_name = "Qwen/Qwen2.5-Coder-0.5B-Instruct"  # يمكن اختيار نموذج آخر من Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)

# تعيين رمز pad_token إلى eos_token إذا لم يكن موجودًا
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# إعداد الإدخال
input_text = "What is Python?"
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# إعداد attention_mask و pad_token_id
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).int()
pad_token_id = tokenizer.pad_token_id

# إعدادات التوليد
generation_kwargs = {
    "max_length": 100,  # زيادة طول النص الناتج
    "num_beams": 5,
    "temperature": 1.2,  # زيادة تنوع النص
    "top_k": 50,
    "top_p": 0.95,  # تحسين جودة النص
    "do_sample": True,
    "repetition_penalty": 2.0,  # تقليل التكرار
    "no_repeat_ngram_size": 3,
    "pad_token_id": pad_token_id
}

# تشغيل الاستدلال بدون xFormers
start_time = time.time()
with torch.no_grad():
    outputs_no_xformers = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], **generation_kwargs)
end_time = time.time()
time_no_xformers = end_time - start_time

# تمكين xFormers
try:
    import xformers.ops

    model_with_xformers = model.half()  # استخدام الدقة النصفية لتحسين الأداء
    model_with_xformers = model_with_xformers.to('cuda')  # نقل النموذج إلى GPU إذا كانت متاحة

    # تشغيل الاستدلال مع xFormers
    start_time = time.time()
    with torch.no_grad():
        outputs_with_xformers = model_with_xformers.generate(inputs['input_ids'].to('cuda'), attention_mask=inputs['attention_mask'].to('cuda'), **generation_kwargs)
    end_time = time.time()
    time_with_xformers = end_time - start_time

    # استخراج النص الناتج
    output_text_with_xformers = tokenizer.decode(outputs_with_xformers[0], skip_special_tokens=True)

except ImportError:
    print("xFormers is not installed. Skipping the optimization step.")
    time_with_xformers = None
    output_text_with_xformers = None

# استخراج النص الناتج بدون xFormers
output_text_no_xformers = tokenizer.decode(outputs_no_xformers[0], skip_special_tokens=True)

# عرض النتائج
print(f"Output without xFormers: {output_text_no_xformers}")
print(f"Time without xFormers: {time_no_xformers:.6f} seconds")

if time_with_xformers is not None:
    print(f"Output with xFormers: {output_text_with_xformers}")
    print(f"Time with xFormers: {time_with_xformers:.6f} seconds")
else:
    print("Skipped xFormers optimization due to missing library.")

In [ ]:
enable_xformers_memory_efficient_attention()

In [ ]:
Note that this is exact attention, not an approximation, just by calling
xformers.ops.memory_efficient_attention

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    use_safetensors=True,
).to("cuda")

pipe.enable_xformers_memory_efficient_attention()

with torch.inference_mode():
    sample = pipe("a small cat")

# optional: You can disable it via
# pipe.disable_xformers_memory_efficient_attention()

In [ ]:
from IPython.display import display
display(sample)

In [ ]:
from IPython.display import display

image = sample.images[0]

display(image)

In [ ]:
pipe.enable_vae_tiling()
pipe.enable_xformers_memory_efficient_attention()
xformers.ops.memory_efficient_attention

### شغال

In [ ]:
from diffusers import DiffusionPipeline
import torch
from IPython.display import display

pipe = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    use_safetensors=True,
).to("cuda")

pipe.enable_xformers_memory_efficient_attention()

with torch.inference_mode():
    sample = pipe("a big cat")

# Access the image from the StableDiffusionPipelineOutput object
image = sample.images[0]

# Display the image
display(image)

In [ ]:
from diffusers import DiffusionPipeline
import torch
from IPython.display import display

pipe = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    use_safetensors=True,
).to("cuda")

pipe.enable_xformers_memory_efficient_attention()

with torch.inference_mode():
    sample = pipe("teen boy")

# Access the image from the StableDiffusionPipelineOutput object
image = sample.images[0]

# Display the image
display(image)

In [ ]:
pipe.enable_xformers_memory_efficient_attention()

with torch.inference_mode():
    sample = pipe("teen boy")

# Access the image from the StableDiffusionPipelineOutput object
image = sample.images[0]

# Display the image
display(image)

In [ ]:
pipe.enable_xformers_memory_efficient_attention()

with torch.inference_mode():
    sample = pipe("crocodile")

# Access the image from the StableDiffusionPipelineOutput object
image = sample.images[0]

# Display the image
display(image)

In [ ]:
pipe.enable_xformers_memory_efficient_attention()

with torch.inference_mode():
    sample = pipe("")

# Access the image from the StableDiffusionPipelineOutput object
image = sample.images[0]

# Display the image
display(image)

In [ ]:
from IPython.display import display

image = sample.images[0]

display(image)

شغال

صح1

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from IPython.display import display

pipe = StableDiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    use_safetensors=True,
)
pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
pipe.enable_vae_slicing()
pipe.enable_xformers_memory_efficient_attention()
images = pipe([prompt] * 32).images




صح2
تكملة

In [ ]:
# Call the pipe with your prompt and assign to sample
sample = pipe(prompt)

# Now you can access the generated image
image = sample.images[0]

display(image)

In [ ]:
image = sample.images[0]

display(image)


Pox Hox
اشرح معنى pipe.enable_vae_slicing() pipe.enable_xformers_memory_efficient_attention()

Gemini
شرح معنى الأوامر:
الأوامر pipe.enable_vae_slicing() و pipe.enable_xformers_memory_efficient_attention() هي طرق لتحسين أداء واستهلاك الذاكرة لعمليات الاستدلال (inference) في مكتبة Diffusers من Hugging Face، وهي تُستخدم عادةً مع نماذج توليد الصور.

1. pipe.enable_vae_slicing()

الوظيفة: تهدف هذه الطريقة إلى تقسيم عملية فك ترميز VAE (Variational Autoencoder) إلى أجزاء أصغر.
الفائدة:
تقليل استهلاك الذاكرة، خاصة عند التعامل مع صور عالية الدقة.
تحسين أداء الاستدلال على الأجهزة ذات ذاكرة VRAM محدودة.
التفاصيل: VAE هو جزء أساسي في نماذج توليد الصور، وهو مسؤول عن تحويل التمثيلات الكامنة (latent representations) إلى صور. تقسيم هذه العملية إلى أجزاء أصغر يُتيح معالجة الصور على مراحل دون تحميلها بالكامل في الذاكرة.
2. pipe.enable_xformers_memory_efficient_attention()

الوظيفة: تمكين استخدام xFormers، وهي مكتبة توفر عمليات انتباه (attention) مُحسّنة للذاكرة.
الفائدة:
تقليل استهلاك الذاكرة بشكل كبير أثناء عمليات الانتباه في النموذج.
تسريع عمليات الاستدلال.
التفاصيل: عمليات الانتباه هي جزء أساسي من العديد من نماذج التعلم العميق، لكنها يمكن أن تستهلك الكثير من الذاكرة. xFormers توفر تطبيقات بديلة لعمليات الانتباه مُحسّنة للذاكرة والأداء.
بإختصار:

pipe.enable_vae_slicing() تُستخدم لتقسيم عملية فك ترميز VAE لتقليل استهلاك الذاكرة.
pipe.enable_xformers_memory_efficient_attention() تُستخدم لتمكين عمليات انتباه مُحسّنة للذاكرة من xFormers.
ملاحظة:

يجب تثبيت مكتبة xFormers أولاً قبل استخدام pipe.enable_xformers_memory_efficient_attention(). يمكنك تثبيتها باستخدام الأمر !pip install xformers.
هذه الطرق مُتاحة في مكتبة Diffusers من Hugging Face.
ستكون هذه الطرق مفيدة بشكل خاص عند استخدام نماذج توليد صور كبيرة أو التعامل مع صور عالية الدقة على

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from IPython.display import display

pipe = StableDiffusionPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-schnell",
    torch_dtype=torch.float16,
    use_safetensors=True,
)
pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
pipe.enable_vae_slicing()
pipe.enable_xformers_memory_efficient_attention()
images = pipe([prompt] * 32).images




In [ ]:
# Call the pipe with your prompt and assign to sample
sample = pipe(prompt)

# Now you can access the generated image
image = sample.images[0]

display(image)

VAE المبلط
تتيح معالجة VAE المتجانب أيضا العمل مع الصور الكبيرة على VRAM محدود (على سبيل المثال ، إنشاء صور بدقة 4K على 8 جيجابايت من VRAM) عن طريق تقسيم الصورة إلى مربعات متداخلة ، وفك تشفير البلاط ، ثم مزج المخرجات معا لإنشاء الصورة النهائية. يجب عليك أيضا استخدام VAE المجانب مع enable_xformers_memory_efficient_attention() لتقليل استخدام الذاكرة بشكل أكبر إذا كان لديك xFormers مثبتا.

لاستخدام معالجة VAE المتجانبة، استدعاء enable_vae_tiling() على المسار قبل الاستدلال:

شغال

ج1

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler

pipe = StableDiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    use_safetensors=True,
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")
prompt = "a beautiful landscape photograph"
pipe.enable_vae_tiling()
#pipe.enable_xformers_memory_efficient_attention()

image = pipe([prompt], width=3840, height=2224, num_inference_steps=20).images[0]

ج2

In [ ]:
from IPython.display import display

# ... (كودك الحالي) ...

# عرض الصورة
display(image)

شغال
ج1

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    use_safetensors=True,
)

prompt = "a photo of an astronaut riding a horse on mars"
pipe.enable_sequential_cpu_offload()
image = pipe(prompt).images[0]

ج2

In [ ]:
from IPython.display import display

# ... (كودك الحالي) ...

# عرض الصورة
display(image)

In [ ]:
print(pipe.unet.conv_out.state_dict()["weight"].stride())  # (2880, 9, 3, 1)
pipe.unet.to(memory_format=torch.channels_last)  # in-place operation
print(
    pipe.unet.conv_out.state_dict()["weight"].stride()
)  # (2880, 1, 960, 320) having a stride of 1 for the 2nd dimension proves that it works

https://huggingface.co/docs/diffusers/optimization/fp16

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

distilled = StableDiffusionPipeline.from_pretrained(
    "nota-ai/bk-sdm-small", torch_dtype=torch.float16, use_safetensors=True,
).to("cuda")
prompt = "a golden vase with different flowers"
generator = torch.manual_seed(2023)
image = distilled("a golden vase with different flowers", num_inference_steps=25, generator=generator).images[0]
image